In [44]:
import json
import torch

from minicons import scorer

from torch.utils.data import DataLoader
from tqdm import tqdm

from sklearn.metrics import accuracy_score, f1_score

In [10]:
def get_anli(split = "dev"):
    labels = []
    with open(f"../data/anli/{split}-labels.lst", "r") as f:
        for line in f:
            labels.append(int(line))
            
    anli = []
    with open(f"../data/anli/{split}.jsonl", "r") as f:
        for line in f:
            data = json.loads(line)
            anli.append([data['obs2'], f"{data['obs1']} {data['hyp1']}", f"{data['obs1']} {data['hyp2']}"])
            
    return anli, labels

In [23]:
anli, labels = get_anli("test")

In [19]:
gpt = scorer.IncrementalLMScorer("gpt2")

Using pad_token, but it is not set yet.


In [30]:
anli_dl = DataLoader(anli, batch_size = 20, num_workers = 4)

In [32]:
hyp1_scores = []
hyp2_scores = []
for batch in tqdm(anli_dl):
    obs2, hyp1, hyp2 = batch
    hyp1_scores.extend(gpt.partial_score(list(hyp1), list(obs2)))
    hyp2_scores.extend(gpt.partial_score(list(hyp2), list(obs2)))
    

100%|██████████| 153/153 [12:50<00:00,  5.03s/it]


In [52]:
predicted = (torch.stack((torch.tensor(hyp1_scores), torch.tensor(hyp2_scores))).argmax(0)+1)

In [56]:
(torch.tensor(labels) == predicted).float().mean()

tensor(0.5273)

In [48]:
len(labels) == len(predicted)

True

In [51]:
accuracy_score(labels, predicted)

0.5272965021248774

In [39]:
torch.tensor(hyp1_scores)

tensor([-2.9011, -3.5036, -3.3421,  ..., -4.0807, -3.2114, -2.7891])

In [27]:
gpt.partial_score(list(hyp1), list(obs2)), gpt.partial_score(list(hyp2), list(obs2))

([-2.789137840270996], [-3.0277607440948486])